# 라이브러리를 사용해서 클러스터링을 해보자

read_data_from_folder

In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster.kmeans import MyKmeans

In [4]:
#kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

#kind_data2 = tools.dict_data_preprocess(kind_data, function=np.sum, window_size='Y',fillna=True)
financial_data2 = tools.dict_data_preprocess(financial_data, function=np.mean, window_size='Y',fillna=False)

c:\Jehyeon\2022_Winter_KRX\KRX\krx_fr\preprocess\tools.py:25: DtypeWarning: Columns (3,4,27,31,36,40,63,65,71,103,104,105,115,121,132,160,179,180,182,187,197,199,204,207,214,217,224,225,226,240,248,257,261,265,275,279,282,301,307,309,312,314,319,324,328,338,347,352,353,360,364,369,372,373,381,387,390,392,393,398,407,412,416,438,443,456,467,468,474,480,484,494,507,509,512,513,525,529,537,540,543,547,558,560,565,579,591,592,594,600,601,606,612,620,634,637,641,647,657,663,667,675,688,690,692,695,702,720,722,731,747,749,768,772,780,787,788,801,825,827,828,829,836,837,839,843,844,854,861,863,880,890,891,896,904,906,910,930,931,932,940,946,948,949,952,956,960,961,978,994,1007,1008,1014,1032,1038,1039,1047,1049,1050,1063,1074,1075,1090,1092,1099,1105,1113,1123,1131,1141,1144,1148,1153,1170,1173,1180,1188,1195,1202,1205,1206,1209,1210,1212,1222,1225,1226,1227,1237,1241,1244,1246,1247,1286,1289,1296,1304,1309,1311,1316,1318,1325,1326,1330,1346,1347,1351,1362,1384,1388,1389,1391,1392,1394,1397,1

KeyError: Timestamp('2012-01-01 00:00:00', freq='AS-JAN')

In [ ]:
kind_data

In [ ]:
dl = tools.dict_data_calculate_vol(financial_data, name="최대주주비율", return_original=True)

In [ ]:
concat_data = tools.dict_data_concat(kind_data2,financial_data2, dropna=False)

print(len(concat_data["2012-01-01"].columns))

In [ ]:
tools.dict_data_calculate_vol(concat_data, "최대주주비율", return_original=True)["2012-01-01"]

In [ ]:
plus_data = tools.dict_data_plus(concat_data, plus_one="BW금액_전체시장",plus_two="CB금액_전체시장", return_origianl=False)

plus_data["2012-01-01"].columns

In [ ]:
plus_data2 = tools.dict_data_plus(plus_data, plus_one="상호변경카운트_전체시장",plus_two="업종변경카운트_전체시장", return_origianl=False)
plus_data2["2012-01-01"].columns

In [ ]:
plus_data3 = tools.dict_data_plus(plus_data2, plus_one="불성실공시법인지정카운트_전체시장",plus_two="조회공시요구카운트_전체시장", return_origianl=False)
plus_data3["2012-01-01"].columns

In [ ]:
plus_data4 = tools.dict_data_plus(plus_data3, plus_one="불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장",plus_two="최대주주변경카운트_전체시장", return_origianl=False)
plus_data4["2012-01-01"].columns

In [ ]:
plus_data5 = tools.dict_data_plus(plus_data4, plus_one="자기주식처분결정금액_전체시장",plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
plus_data5["2012-01-01"].columns

In [ ]:
divide_data = tools.dict_data_divide(plus_data5,divide_what=['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '판매 및 관리비'], divide_into="총자본", return_origianl=False)
print(len(divide_data["2012-01-01"].columns))
divide_data["2012-01-01"].columns

In [ ]:
drop_data = tools.dict_data_drop(divide_data, dropna=True)
scaled_data = tools.dict_data_scale(drop_data, scaler=scaler.sklearn_maxabs)

In [ ]:
scaled_data["2022-01-01"].shape

In [ ]:
# 클러스터링 할 sample data 하나 생성
sample_2022 = scaled_data["2022-01-01"]
company = scaled_data["2022-01-01"].index.to_list()

In [ ]:
sample_2022.columns

In [ ]:
import pandas as pd
df = pd.DataFrame({"a":[1,2,3,4,5], "b":[10,20,30,40,50], "c":[0,0,0,0,0]})
scaler.sklearn_minmax(df)

In [ ]:
import matplotlib.pyplot as plt 

plt.figure(figsize=(20,12))
plt.scatter(range(len(sample_2022["최대주주비율"].values)),sample_2022["최대주주비율"].values);

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, n_init=100, max_iter=10000)
kmeans.fit(sample_2022)

cluster_df = pd.DataFrame(data=np.c_[company,kmeans.labels_], columns=["기업공시코드","클러스터"])
cluster_df["클러스터"].value_counts()

거리요소로 분해하기 위해선 kmeans를 써주는 게 중요하다

## 클러스터링 라이브러리

In [ ]:
cluster_model = MyKmeans(sample_2022)
cluster_model.set_params()
cluster_model.find_optimal_k(max_k=10, max_sample=20, optimize_method="silhouette")
#cluster_model.run_kmean(5,)
#cluster_model.

In [ ]:
cluster_model.find_optimal_initp(5,20, "silhouette")

In [ ]:
k_mean_dict = cluster_model.run_kmean()

In [ ]:
cluster_model.distance_decomposition(k_mean_dict["model"])[0][1]

In [ ]:
for i, score_list in dict_sil.items():
    if i == 0:
        start_ar = np.array(score_list)
    elif i ==1:
        concat_ar = np.c_[start_ar, score_list]
    else:
        concat_ar = np.c_[concat_ar,score_list]

mean_ar = concat_ar.mean(axis=1)
print(mean_ar)
print("최고 점수의 인덱스는: ", mean_ar.argmax())

In [ ]:
for i, score in dict_sil.items():
    ar = np.array(score)
    print("{}번째 최대값의 인덱스는 ")

In [ ]:
test = MyKmeans.distance_decomposition(kmeans)

In [ ]:
test[0][1]

In [ ]:
MyKmeans.visualize(sample_2022,kmeans)

In [ ]:
# 클러스터링을 해보자
from sklearn.cluster import AgglomerativeClustering

agg = AgglomerativeClustering(n_clusters=5)
agg.fit(sample_2022)

clus = pd.DataFrame(np.c_[company,agg.labels_], columns= ["기업공시코드","클러스터"])
clus["클러스터"].value_counts()

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, n_init=100, max_iter=10000)
kmeans.fit(sample_2022)

cluster_df = pd.DataFrame(data=np.c_[company,kmeans.labels_], columns=["기업공시코드","클러스터"])
cluster_df["클러스터"].value_counts()

In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN(min_samples=40, n_jobs=-1)
db.fit(sample_2022)

label_db = db.labels_
cluster_db = pd.DataFrame(data=np.c_[company, label_db], columns=["기업공시코드","클러스터"])
cluster_db["클러스터"].value_counts()